In [1]:
# 查询测试

import MySQLdb
import elasticsearch
from elasticsearch.helpers import bulk, streaming_bulk
from IPython.display import display, HTML
from elasticsearch_dsl import F, Q, Search

es = elasticsearch.Elasticsearch('http://172.16.1.30:9200')
INDEX = 'meila_wares'
TYPE = 'wares'


#q = raw_input('请输入关键词：')
q = "美白"


if not q: q = u'美白 面膜'

s = Search(es, index=INDEX, doc_type=TYPE)

# 查询 DSL
s = s.query(    
Q('bool', should=[Q('constant_score', boost=1.0, query=Q('match', name=dict(query=q, operator='or')))
    , Q('constant_score', boost=0.2, query=Q('match', short_name=dict(query=q, operator='or')))
    , Q('constant_score', boost=0.1, query=Q('match', short_intro=dict(query=q, operator='or')))], disable_coord=True)
)

s = s.params(explain=True)

# 排序
s = s.sort({"_script": {
            "script": "_score * 68.0 + doc['display_score'].value * 0.5 + doc['sale_score'].value * 0.3 + doc['comment_score'].value * 0.2",
            "type": "number", "order": "desc", "lang": "groovy"}})

# 分页
s = s[:200]


display(s.to_dict())

r = s.execute()

htmls = [u'<br><br>'
         u'<table>'
        u'<tr>'
        u'<th>#</th>'
        u'<th>ID</th>'
        u'<th>标题</th>'
        u'<th>短标题</th>'
        u'<th>短描述</th>'
        u'<th>销量分数</th>'
        u'<th>点评分数</th>'
        u'<th>设置的分数</th>'
        u'<th>ES分数 * 68.0</th>'
        u'<th>ES分数</th>'
        u'<th>最后的分数</th>'
         u'<th>解释</th>'
       ]

for idx, doc in enumerate(r):
     htmls.extend([u'<tr>'
                  u'<td>{idx}</td>'.format(idx=idx+1),
                  u'<td>{id}</td>'.format(id=doc.id),
                  u'<td>{name}</td>'.format(name=doc.name),
                  u'<td>{short_name}</td>'.format(short_name=doc.short_name),
                  u'<td>{short_intro}</td>'.format(short_intro=doc.short_intro),
                  u'<td>{sale_score}</td>'.format(sale_score=doc.sale_score),
                  u'<td>{comment_score}</td>'.format(comment_score=doc.comment_score),
                  u'<td>{display_score}</td>'.format(display_score=doc.display_score),
                  u'<td>{es_score}</td>'.format(es_score=(doc.meta.sort[0] - (doc.display_score * 0.5 + doc.sale_score * 0.3 + doc.comment_score * 0.2))),
                  u'<td>{es_score}</td>'.format(es_score=(doc.meta.sort[0] - (doc.display_score * 0.5 + doc.sale_score * 0.3 + doc.comment_score * 0.2))/68.0),
                  u'<td>{last_score}</td>'.format(last_score=doc.meta.sort[0]),
                  u'<td>{explanation}</td>'.format(explanation=', '.join([str(x.value) for x in doc.meta.explanation.details]))]
                 )

display(HTML(''.join(htmls)))

{'from': 0,
 'query': {'bool': {'disable_coord': True,
   'should': [{'constant_score': {'boost': 1.0,
      'query': {'match': {'name': {'operator': 'or',
         'query': '\xe7\xbe\x8e\xe7\x99\xbd'}}}}},
    {'constant_score': {'boost': 0.2,
      'query': {'match': {'short_name': {'operator': 'or',
         'query': '\xe7\xbe\x8e\xe7\x99\xbd'}}}}},
    {'constant_score': {'boost': 0.1,
      'query': {'match': {'short_intro': {'operator': 'or',
         'query': '\xe7\xbe\x8e\xe7\x99\xbd'}}}}}]}},
 'size': 200,
 'sort': [{'_script': {'lang': 'groovy',
    'order': 'desc',
    'script': "_score * 68.0 + doc['display_score'].value * 0.5 + doc['sale_score'].value * 0.3 + doc['comment_score'].value * 0.2",
    'type': 'number'}}]}

#,ID,标题,短标题,短描述,销量分数,点评分数,设置的分数,ES分数 * 68.0,ES分数,最后的分数,解释
1,692,【海淘专场】韩国su:m 37 三合一美白排毒面膜小样5ml *10片<60包邮>,三合一美白排毒面膜,排毒养颜神器,90,100,0,79.6334476471,1.17108011246,126.633447647,"0.9759001, 0.19518003"
2,498,【爆款】台湾SHILLS舒儿丝很耐晒超清爽美白防晒冰镇喷雾SPF50 180ml<79包邮>,舒儿丝美白防晒喷雾,《女人我最大》强力推荐,84,100,0,79.6334476471,1.17108011246,124.833447647,"0.9759001, 0.19518003"
3,957,【海淘专场】日本 白兔牌HYTHIOL-C 美白丸加强版30日/180粒 <178包邮>,白兔牌美白丸,让自己变白的必备选择,99,70,0,79.6334476471,1.17108011246,123.333447647,"0.9759001, 0.19518003"
4,782,【周末卖场】中国 御泥坊自然美白嫩肤蚕丝面膜14片 30ml/片<79.9包邮>,美白嫩肤蚕丝面膜,销量第一的蚕丝面膜,93,74,0,79.6334476471,1.17108011246,122.333447647,"0.9759001, 0.19518003"
5,975,【海淘专场】 日本 SHISEIDO 资生堂HAKU美白淡斑深层保湿化妆水 120ml<289包邮>,资生堂美白淡斑保湿水,一白遮百丑,99,64,0,79.6334476471,1.17108011246,122.133447647,"0.9759001, 0.19518003"
6,977,【海淘专场】 日本 SHISEIDO 资生堂HAKU美白淡斑深层保湿乳 120ml<328包邮>,资生堂美白淡斑保湿乳,夏日静静的白,99,64,0,79.6334476471,1.17108011246,122.133447647,"0.9759001, 0.19518003"
7,3389,韩国 Phyto Tree美白补水晒后修护黄瓜胶 250ML 35包邮,美白补水,晒后修护黄瓜胶 250ML 35包邮,100,60,0,79.6334476471,1.17108011246,121.633447647,"0.9759001, 0.19518003"
8,945,【海淘专场】日本资生堂CPB美白防晒妆前隔离乳 spf35 30ml <428包邮>,CPB美白防晒隔离乳,潮女必备防晒隔离乳,98,62,0,79.6334476471,1.17108011246,121.433447647,"0.9759001, 0.19518003"
9,995,【海淘专场】韩国Amore/爱茉莉 麦迪安 86%美白护齿健康牙龈两支一组<40包邮>,爱茉莉 86 牙膏,美白护齿健康牙龈,100,74,0,72.9973297119,1.07349014282,117.797329712,"0.9759001, 0.097590014"
10,2882,韩国VITASNOW白雪公主微整医院美白粉1精华30ml+4美白粉5ml 368包邮,JAYJUN美白粉,原装天然美白祛斑祛印,63,60,0,86.2695736885,1.2686702013,117.169573689,"0.9759001, 0.19518003, 0.097590014"
